In [1]:
"""
LoL Feature 상관관계 분석 (라인별 분리 분석)
- 시각화 스타일 개선: 삼각형 히트맵, 분 단위 시계열 그래프 적용
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
import platform
import os

# ============================================================
# 설정
# ============================================================
DATA_PATH = "data/final_features.csv"
BASE_OUTPUT_DIR = Path("output/correlation_analysis")

# 분석할 Feature 목록
FEATURE_COLS = [
    'xp', 'level', 'minionsKilled', 'jungleMinionsKilled',
    'totalDamageDoneToChampions', 'totalDamageTaken', 'timeEnemySpentControlled',
    'kills', 'deaths', 'assists', 'soloKills',
    'turretPlates', 'wardsPlaced', 'objectiveParticipation',
    'xpDiff', 'csDiff', 'damageDiff', 'killsDiff', 'levelDiff',
    'KDA', 'killParticipation', 'damageShare'
]

TARGET_COL = 'totalGold'

# ============================================================
# 유틸리티 함수
# ============================================================

def set_korean_font():
    """운영체제에 맞는 한글 폰트 설정"""
    system_name = platform.system()
    if system_name == 'Darwin':  # Mac
        plt.rcParams['font.family'] = 'AppleGothic'
    elif system_name == 'Windows':  # Windows
        plt.rcParams['font.family'] = 'Malgun Gothic'
    else:  # Linux
        plt.rcParams['font.family'] = 'NanumGothic'
    plt.rcParams['axes.unicode_minus'] = False

def load_data(path: str) -> pd.DataFrame:
    print(f"Loading data from {path}...")
    if not os.path.exists(path):
        print(f"Error: 파일이 존재하지 않습니다. ({path})")
        print("01_5_preprocess_duo.py를 먼저 실행해주세요.")
        exit(1)

    df = pd.read_csv(path)
    df = df[df['minute'] > 0].reset_index(drop=True)
    return df

# ============================================================
# 분석 함수들
# ============================================================

def analyze_feature_target_correlation(df: pd.DataFrame, output_dir: Path):
    print("   [1] Feature-Target Correlation...")
    corrs = []
    for col in FEATURE_COLS:
        if col in df.columns:
            if df[col].std() == 0: continue
            r, p = stats.pearsonr(df[col], df[TARGET_COL])
            corrs.append({'Feature': col, 'Correlation': r, 'P-value': p})

    if not corrs: return pd.DataFrame()

    res_df = pd.DataFrame(corrs).sort_values('Correlation', ascending=False)
    res_df.to_csv(output_dir / "feature_target_correlation.csv", index=False)
    return res_df

from statsmodels.stats.outliers_influence import variance_inflation_factor

def analyze_multicollinearity(df: pd.DataFrame, output_dir: Path):
    """
    다중공선성(VIF) 분석 및 Correlation Matrix 계산
    - VIF가 10 이상이면 다중공선성이 높음 (서로 겹치는 변수)
    """
    # print("   [2] Multicollinearity Analysis (VIF)...")

    # 1. 상관관계 매트릭스용 데이터
    valid_cols = [c for c in FEATURE_COLS if c in df.columns and df[c].std() > 0]
    corr_matrix = df[valid_cols].corr()

    # 2. VIF 계산
    # 결측치 제거 및 무한대 값 방지
    vif_df = df[valid_cols].dropna()
    vif_data = pd.DataFrame()
    vif_data["Feature"] = valid_cols

    try:
        # VIF 계산 (데이터가 너무 많으면 시간이 걸릴 수 있어 샘플링 가능)
        if len(vif_df) > 10000:
            vif_df = vif_df.sample(10000, random_state=42)

        vif_data["VIF"] = [variance_inflation_factor(vif_df.values, i)
                          for i in range(len(valid_cols))]

        # VIF 높은 순서대로 정렬 및 저장
        vif_data = vif_data.sort_values(by="VIF", ascending=False)
        vif_data.to_csv(output_dir / "vif.csv", index=False)
    except Exception as e:
        print(f"   ⚠️ VIF Calculation Failed: {e}")
        # 실패하더라도 corr_matrix는 반환해야 함

    return corr_matrix

def analyze_correlation_by_time_detailed(df: pd.DataFrame, output_dir: Path):
    """
    시간대별(분 단위) 상관관계 변화 분석
    구간(Bin)이 아닌 Minute별 상관계수를 직접 계산합니다.
    """
    print("   [3] Time-based Correlation (Detailed)...")

    # 분 단위 그룹화 (데이터가 적은 0분, 40분 이후 등은 제외 가능)
    # 여기서는 1분~35분 정도까지만 유효한 데이터로 봅니다.
    max_minute = min(df['minute'].max(), 40)
    minutes = range(1, int(max_minute) + 1)

    results = []

    for m in minutes:
        minute_df = df[df['minute'] == m]
        # 데이터가 너무 적으면 스킵 (최소 10개 게임 이상)
        if len(minute_df) < 10: continue

        for col in FEATURE_COLS:
            if col in minute_df.columns and minute_df[col].std() > 0:
                r = minute_df[col].corr(minute_df[TARGET_COL])
                results.append({'Minute': m, 'Feature': col, 'Correlation': r})

    if not results: return pd.DataFrame()

    time_corr_df = pd.DataFrame(results)

    # Pivot: Index=Feature, Columns=Minute
    pivot_df = time_corr_df.pivot(index='Feature', columns='Minute', values='Correlation')

    # 저장
    pivot_df.to_csv(output_dir / "correlation_by_time.csv")
    return pivot_df

def analyze_correlation_by_win(df: pd.DataFrame, output_dir: Path):
    print("   [4] Win/Loss Correlation...")
    win_df = df[df['win'] == True]
    lose_df = df[df['win'] == False]

    results = []
    for col in FEATURE_COLS:
        if col in df.columns and win_df[col].std() > 0 and lose_df[col].std() > 0:
            r_win = win_df[col].corr(win_df[TARGET_COL])
            r_lose = lose_df[col].corr(lose_df[TARGET_COL])
            results.append({
                'Feature': col,
                'Win_Corr': r_win,
                'Lose_Corr': r_lose,
                'Diff': abs(r_win - r_lose)
            })

    res_df = pd.DataFrame(results).sort_values('Diff', ascending=False)
    res_df.to_csv(output_dir / "correlation_by_win.csv", index=False)
    return res_df

# ============================================================
# 시각화 함수들 (수정된 부분)
# ============================================================

def plot_correlation_matrix_triangle(corr_matrix, output_path, position_name):
    """
    [수정됨] 삼각형(Lower Triangle) 히트맵
    """
    if corr_matrix.empty: return

    plt.figure(figsize=(14, 12))

    # 위쪽 삼각형을 가리기 위한 마스크 생성
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

    # 히트맵 그리기
    sns.heatmap(corr_matrix,
                mask=mask,              # 마스크 적용
                annot=False,            # 숫자는 너무 복잡하므로 끔
                cmap='coolwarm',
                center=0,
                vmin=-1, vmax=1,
                square=True,            # 정사각형 비율 유지
                linewidths=.5,          # 격자 간격
                cbar_kws={"shrink": .5}) # 컬러바 크기 조절

    plt.title(f'Feature Correlation Matrix - {position_name}', fontsize=16)
    plt.tight_layout()
    plt.savefig(output_path, dpi=150)
    plt.close()

def plot_top_correlations(corr_df, output_path, position_name):
    if corr_df.empty: return
    plt.figure(figsize=(10, 8))

    # 상위 10개, 하위 5개
    top_pos = corr_df.head(10)
    top_neg = corr_df.tail(5)
    plot_data = pd.concat([top_pos, top_neg])

    sns.barplot(x='Correlation', y='Feature', data=plot_data, palette='coolwarm')
    plt.title(f'Top Features vs Gold - {position_name}', fontsize=15)
    plt.axvline(0, color='black', linewidth=0.8)
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_path, dpi=150)
    plt.close()

def plot_time_correlation_lines(pivot_df, output_path, position_name):
    """
    [수정됨] 시간대별 상관관계를 라인 차트로 시각화 (중요 Feature 위주)
    """
    if pivot_df is None or pivot_df.empty: return

    plt.figure(figsize=(14, 8))

    # 너무 많은 선은 복잡하므로 상관관계가 높은 주요 Feature만 필터링해서 그림
    # 예: 마지막 시간대 기준 절대값 상관계수가 높은 상위 10개
    last_col = pivot_df.columns[-1]
    top_features = pivot_df[last_col].abs().nlargest(10).index

    plot_data = pivot_df.loc[top_features].T # 행/열 전환 (Index: Minute)

    # 라인 차트 그리기
    sns.lineplot(data=plot_data, dashes=False, palette='tab10', linewidth=2)

    plt.title(f'Correlation Trends over Time (Top 10 Features) - {position_name}', fontsize=16)
    plt.xlabel('Minute', fontsize=12)
    plt.ylabel('Correlation with Gold', fontsize=12)
    plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_path, dpi=150)
    plt.close()

def plot_time_correlation_heatmap_detailed(pivot_df, output_path, position_name):
    """
    [수정됨] 시간대별 상관관계를 띠 형태의 히트맵으로 시각화 (전체 Feature)
    X축: 시간(분), Y축: Feature
    """
    if pivot_df is None or pivot_df.empty: return

    plt.figure(figsize=(15, 10))

    # 상관관계 총합이 높은 순서로 정렬 (시각적으로 보기 좋게)
    sorted_idx = pivot_df.mean(axis=1).abs().sort_values(ascending=False).index
    sorted_df = pivot_df.loc[sorted_idx]

    sns.heatmap(sorted_df,
                annot=False,
                cmap='RdBu_r', # 빨강-파랑 (Red=양의상관, Blue=음의상관)
                center=0,
                vmin=-1, vmax=1,
                cbar_kws={'label': 'Correlation'})

    plt.title(f'Feature Importance (Correlation) over Time - {position_name}', fontsize=16)
    plt.xlabel('Minute')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig(output_path, dpi=150)
    plt.close()


# ============================================================
# 메인 실행
# ============================================================

def main():
    set_korean_font()
    df = load_data(DATA_PATH)

    positions = df['position'].unique()
    print(f"Detected Positions: {positions}")

    for pos in positions:
        print(f"\n" + "="*50)
        print(f"🚀 Analyzing Position: {pos}")
        print("="*50)

        pos_df = df[df['position'] == pos].copy()

        if len(pos_df) < 100:
            print(f"Skipping {pos}: Not enough data rows ({len(pos_df)})")
            continue

        pos_output_dir = BASE_OUTPUT_DIR / pos
        pos_output_dir.mkdir(parents=True, exist_ok=True)

        # 1. Feature-Target 분석
        corr_result = analyze_feature_target_correlation(pos_df, pos_output_dir)

        # 2. 다중공선성 분석
        corr_matrix = analyze_multicollinearity(pos_df, pos_output_dir)

        # 3. 시간대별 분석 (분 단위 상세)
        time_corr_df = analyze_correlation_by_time_detailed(pos_df, pos_output_dir)

        # 4. 승패별 분석
        analyze_correlation_by_win(pos_df, pos_output_dir)

        # 5. 시각화 생성
        print("   Generating Plots...")

        # [변경 1] 삼각형 히트맵
        plot_correlation_matrix_triangle(corr_matrix, pos_output_dir / "correlation_matrix.png", pos)

        # 막대 그래프
        plot_top_correlations(corr_result, pos_output_dir / "top_features_bar.png", pos)

        # [변경 2] 시간대별 상세 히트맵 (띠 그래프)
        plot_time_correlation_heatmap_detailed(time_corr_df, pos_output_dir / "time_correlation_heatmap.png", pos)

        # [변경 3] 시간대별 라인 차트 (추세선)
        plot_time_correlation_lines(time_corr_df, pos_output_dir / "time_correlation_lines.png", pos)

        print(f"✅ {pos} Analysis Complete! Saved to {pos_output_dir}")

    print("\n" + "="*50)
    print(f"All analyses finished. Check {BASE_OUTPUT_DIR}")

if __name__ == "__main__":
    main()

Loading data from data/final_features.csv...
Detected Positions: ['BOT_DUO' 'JUNGLE' 'MIDDLE' 'TOP']

🚀 Analyzing Position: BOT_DUO
   [1] Feature-Target Correlation...
   [3] Time-based Correlation (Detailed)...
   [4] Win/Loss Correlation...
   Generating Plots...


/var/folders/3r/fkfjb6j92yl5dmn63k5ggz4m0000gn/T/ipykernel_29095/2992613453.py:210: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Correlation', y='Feature', data=plot_data, palette='coolwarm')


✅ BOT_DUO Analysis Complete! Saved to output/correlation_analysis/BOT_DUO

🚀 Analyzing Position: JUNGLE
   [1] Feature-Target Correlation...
   [3] Time-based Correlation (Detailed)...
   [4] Win/Loss Correlation...
   Generating Plots...


/var/folders/3r/fkfjb6j92yl5dmn63k5ggz4m0000gn/T/ipykernel_29095/2992613453.py:210: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Correlation', y='Feature', data=plot_data, palette='coolwarm')


✅ JUNGLE Analysis Complete! Saved to output/correlation_analysis/JUNGLE

🚀 Analyzing Position: MIDDLE
   [1] Feature-Target Correlation...
   [3] Time-based Correlation (Detailed)...
   [4] Win/Loss Correlation...
   Generating Plots...


/var/folders/3r/fkfjb6j92yl5dmn63k5ggz4m0000gn/T/ipykernel_29095/2992613453.py:210: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Correlation', y='Feature', data=plot_data, palette='coolwarm')


✅ MIDDLE Analysis Complete! Saved to output/correlation_analysis/MIDDLE

🚀 Analyzing Position: TOP
   [1] Feature-Target Correlation...
   [3] Time-based Correlation (Detailed)...
   [4] Win/Loss Correlation...
   Generating Plots...


/var/folders/3r/fkfjb6j92yl5dmn63k5ggz4m0000gn/T/ipykernel_29095/2992613453.py:210: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Correlation', y='Feature', data=plot_data, palette='coolwarm')


✅ TOP Analysis Complete! Saved to output/correlation_analysis/TOP

All analyses finished. Check output/correlation_analysis
